<a href="https://colab.research.google.com/github/Zhengro/DL-Identification/blob/narrow-the-exhaustive-search-range/train%20a%20mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import commpy.channels
import matlab.engine
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

eng = matlab.engine.start_matlab()

b = 0  # value of frozen bits
R = 0.5  # compression rate
D = eng.hbinv(1-R)  # corresponding distortion

MM = np.array([[500]])  # number of users
# MM = np.linspace(500, 2000, num=4)

bsc_e = np.array([[0.3]])  # crossover probability of BSC
# bsc_e = np.linspace(0.1, 0.4, num=7)

N = np.array([[256]])  # data length
# N = np.array([[64, 128, 256]])  

# find error rate
for im in range(MM.size):
    M = MM[0, im]
    err_mtx = np.zeros((bsc_e.size, N.size))
    for ie in range(bsc_e.size):
        e = bsc_e[0, ie]
        err = np.zeros((1, N.size))
        for i in range(N.size):
            L = N[0, i]
            K = int(np.round(L*R))

            Z = eng.get_bec_bhattacharyya(float(L), eng.h_2(D))
            index = np.argsort(Z).ravel()
            index = index[::-1]  # sort Z descend using 2 steps
            free_index = index[L-K:L]+1  # messages indices ‘+1’ is to be adapted for MATLAB
            frozen_index = index[0:L-K]+1  # frozen indices
            free_index = matlab.int64(free_index.tolist())  # preparation
            frozen_index = matlab.int64(frozen_index.tolist())

            data_u = np.zeros((M, L))  # original data
            data_y = np.zeros((M, L))  # compressed data
            data_y2 = np.zeros((M, L))  # reconstruction of compressed data
            data_z = np.zeros((M, L))  # observation
            h_y = np.zeros((1, M))
            h_z = np.zeros((1, M))
            I_xz = np.zeros((1, M))
            w_hat = np.zeros((1, M))

            for j in range(M):
                data_u[j, :] = np.random.rand(1, L) <= 0.5
                temp = eng.sc_SC(matlab.double(data_u[j, :].tolist()), free_index, frozen_index, D)
                data_y[j, :] = np.asarray(temp).ravel()

                temp = eng.channel_encoder(matlab.double(data_y[j, :].tolist()))
                data_y2[j, :] = np.asarray(temp).ravel()

                h_y[0, j] = eng.h_2(float(np.mean(data_y2[j, :])))
            for j in range(M):
                data_z[j, :] = commpy.channels.bsc(np.int_(data_u[j, :]), e)
        # MMI identification method
        #         temp = matlab.double(data_z[j, :].tolist())
        #         h_z[0, j] = eng.h_2(float(np.mean(data_z[j, :])))
        #         for k in range(M):
        #             h_xz = eng.get_type_entropy(temp, matlab.double(data_y2[k, :].tolist()))
        #             I_xz[0, k] = h_y[0, k] + h_z[0, j] - h_xz
        #         w_hat[0, j] = np.argmax(I_xz)
        #         if w_hat[0, j] != j:
        #             err[0, i] = err[0, i] + 1
        # err = err / M
        # err_mtx[ie, :] = err

eng.quit()

# clustering
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(data_u)
cluster_label = kmeans.labels_.tolist()
centroids = kmeans.cluster_centers_
plt.hist(cluster_label, bins=num_clusters)
plt.title('Histogram with cluster_label')
plt.show()


def dec_to_bin(x, n):
    idx = len(x)
    y = np.zeros((idx, n))
    for i in range(idx):
        temp_idx = int(x[i])
        y[i, temp_idx] = 1
    return y


# construct train set and test set
ns = [800, 200]  # ns is the number of noisy samples for each users [train, test]
t1 = np.zeros((500, ns[0], 256))
t2 = np.zeros((500, ns[1], 256))
a1 = np.zeros((500, ns[0], 1))
a2 = np.zeros((500, ns[1], 1))
for j in range(500):
    for i in range(ns[0]):
        t1[j, i, :] = commpy.channels.bsc(np.int_(data_u[j, :]), e)
        a1[j, i, :] = cluster_label[j]
    for i in range(ns[1]):
        t2[j, i, :] = commpy.channels.bsc(np.int_(data_u[j, :]), e)
        a2[j, i, :] = cluster_label[j]
train = t1.reshape((500*ns[0], 256))  
test = t2.reshape((500*ns[1], 256))  
train_target = a1.reshape((500*ns[0], 1)).ravel()  
test_target = a2.reshape((500*ns[1], 1)).ravel() 
bin_train_target = dec_to_bin(train_target, num_clusters)  

In [0]:
import numpy as np
# data generated as above
train = np.loadtxt(open('train.csv', 'rb'), delimiter=',', skiprows=0)  # 400000*256
bin_train_target = np.loadtxt(open('bin_train_target.csv', 'rb'), delimiter=',', skiprows=0)  # 400000*5
test = np.loadtxt(open('test.csv', 'rb'), delimiter=',', skiprows=0)  # 100000*256
test_target = np.loadtxt(open('test_target.csv', 'rb'), delimiter=',', skiprows=0)  # 100000*1



In [24]:
from __future__ import print_function
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping

# parameters
epoch = 100
batch_size = 3200
layer1_dim = 120
layer2_dim = 80
layer3_dim = 50

input_img = Input(shape=(256, ))
encoded1 = Dense(layer1_dim, activation='relu')(input_img)
encoded2 = Dense(layer2_dim, activation='relu')(encoded1)
encoded3 = Dense(layer3_dim, activation='relu')(encoded2)
out = Dense(5, activation='sigmoid')(encoded3)

mlp = Model(input_img, out)
mlp.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
mlp.fit(train, bin_train_target, epochs=epoch, batch_size=batch_size, shuffle=True)

test_preds = mlp.predict(test)
predictions = np.argmax(test_preds, axis=1)
diff = predictions - test_target
errate = np.count_nonzero(diff)/len(predictions)
print("Error rate:%s" % errate)


Epoch 1/100
400000/400000 [==============================] - 5s 13us/step - loss: 1.4298 - acc: 0.3955
Epoch 2/100
400000/400000 [==============================] - 4s 10us/step - loss: 1.2375 - acc: 0.5005
Epoch 3/100
400000/400000 [==============================] - 4s 10us/step - loss: 1.1837 - acc: 0.5266
Epoch 4/100
400000/400000 [==============================] - 4s 10us/step - loss: 1.1254 - acc: 0.5571
Epoch 5/100
400000/400000 [==============================] - 4s 10us/step - loss: 1.0536 - acc: 0.5916
Epoch 6/100
400000/400000 [==============================] - 4s 10us/step - loss: 0.9895 - acc: 0.6203
Epoch 7/100
400000/400000 [==============================] - 4s 10us/step - loss: 0.9301 - acc: 0.6463
Epoch 8/100
400000/400000 [==============================] - 4s 10us/step - loss: 0.8801 - acc: 0.6676
Epoch 9/100
400000/400000 [==============================] - 4s 10us/step - loss: 0.8274 - acc: 0.6894
Epoch 10/100
400000/400000 [==============================] - 4s 10us/ste